In [92]:
#Cleaning to-dos:
#clean up mocktest()
#write docstrings
#remove confusing comments


from astropy.coordinates import ICRS
from astropy import units as u
from astropy.coordinates.angles import Angle
from astropy.coordinates import match_coordinates_sky
import numpy as np
import os
import sys
import pandas as pd

class OpenCluster():

    '''
    Open cluster class.
    '''

    def __init__(self, name, sampedro_name, radius=0., age=0., distance=100.):

        #prompt and file specific cluster name
        self.name=name
        self.sampedro_name=sampedro_name
        #radius in acrmin
        self.radius=radius
        #age in Gyr
        self.age=age
        #distance in pc
        self.distance=distance
        #subcats with different numbers of membership allocations
        self.sampedro=[]
        self.sampedro_n1=[]
        self.sampedro_n2=[]
        self.sampedro_n3=[]
        #Pan-STARRS catalog
        self.PS=[]
        #K2 EVEREST search for the cluster 
        self.K2=[]
        #subcat of self.K2 which has 2MASS IDs
        self.K2MASS=[]

    #----------------------------------------------------------------------------------------------

    def loadcatalogs(self,debug=False):

        #path="/work1/eilin/data/CLUSTERS_01/k2-panstarrs-sampedro"
        path='/home/ekaterina/Documents/Matching_Catalogs'
        os.chdir(path)
    #SAMPEDRO
        self.sampedro = pd.read_csv(path+'/cats/'+self.sampedro_name+'_Sampedro_cluster_members_query.csv',
                                       usecols=[2,3,5,57,58,59],
                                       delimiter='\t',
                                      )

        self.sampedro['RAJ2000'] = to_xms(self.sampedro,'RAJ2000','udeg')
        self.sampedro['DEJ2000'] = to_xms(self.sampedro,'DEJ2000','udeg')
         
    #PanSTARRS:
#         self.PS = pd.read_csv(path+'/cats/'+self.sampedro_name+'_panstarrs_search.txt',
#                                        usecols=['objID','raMean','decMean','gMeanPSFMag','rMeanPSFMag','iMeanPSFMag','zMeanPSFMag','yMeanPSFMag'],
#                                        skiprows=[1],
#                                        delimiter='\t',
#                                       )

#         self.PS.rename(index=int,
#                        columns={'raMean':'RAJ2000','decMean':'DEJ2000'},
#                        inplace=True)
#         self.PS['RAJ2000'] = to_xms(self.PS,'RAJ2000','h')
#         self.PS['DEJ2000'] = to_xms(self.PS,'DEJ2000','d')
        self.PS = pd.read_csv(path+'/cats/'+self.sampedro_name+'_ps_vs_ucac4.csv',
                                       usecols=['UCAC4','gmag','rmag','imag','zmag','ymag'],
                                      )

    #K2
        self.K2 = pd.read_csv(path+'/cats/'+self.sampedro_name+'_k2_search.txt',
                                       usecols=['K2 ID','RA (J2000)','Dec (J2000)','2MASS ID','UCAC ID'],
                                       skiprows=[1],
                                       delimiter='\t',
                                      )
        self.K2.rename(index=int,
                       columns={'K2 ID':'EPIC','RA (J2000)':'RAJ2000','Dec (J2000)':'DEJ2000','2MASS ID':'2MASS','UCAC ID':'UCAC4'},
                       inplace=True)
        self.K2['RAJ2000'] = to_xms(self.K2,'RAJ2000','h')
        self.K2['DEJ2000'] = to_xms(self.K2,'DEJ2000','d')
        
    #K2MASS
        self.K2MASS = self.K2[self.K2['2MASS'].notnull()]

        if debug == True:
            print('Currently working in \"' + os.getcwd()+ '\"\n')
            print('These are our K2 data:\n\n{}\n'.format(self.K2.head()))
            print('These are our Sampedro data:\n\n{}\n'.format(self.sampedro.head()))
            print('These are our Pan-STARRS data:\n\n{}\n'.format(self.PS.head()))
            print('\nNumber of Everest LCs: \n{}\n'.format(len(self.K2)))
            print('\nNumber of Everest LCs with 2MASS ID: \n{}\n'.format(len(self.K2MASS)))		

        return

    def refinesampedro(self,debug=False):

        self.sampedro_n1=self.sampedro[(self.sampedro.ClassM1+self.sampedro.ClassM2+self.sampedro.ClassM3)==1]
        self.sampedro_n2=self.sampedro[(self.sampedro.ClassM1+self.sampedro.ClassM2+self.sampedro.ClassM3)==2]
        self.sampedro_n3=self.sampedro[(self.sampedro.ClassM1+self.sampedro.ClassM2+self.sampedro.ClassM3)==3]

        if debug==True:
            print('These are single validation members:\n\n{}\n'.format(self.sampedro_n1.head()))
            print('These are double validation members:\n\n{}\n'.format(self.sampedro_n2.head()))
            print('These are triple validation members:\n\n{}\n'.format(self.sampedro_n3.head()))

        return


        #--------------------------------------------------------------------------------------------
    def readselect(self,ids,cat='PS'):

        import linecache
        cats={'PS':['_panstarrs_search.txt',[24,32,40,48,56]], '2MASS':'_k2_search.txt'}
        dt=np.unicode_
        line=linecache.getline('cats/'+self.sampedro_name+cats[cat][0],ids)
        #print(line)
        myarray = np.fromstring(line,dtype=dt, sep='\t')

        #print(myarray)
        mags=[myarray[i] for i in cats[cat][1]]
        return mags

    #--------------------------------------------------------------------------------------------

    def read_params(self, udict):

        for key, value in udict.items():
            #For all Pan-STARRS IDs:
            if type(value)==np.int64:
                ps=list(self.PS)
                ids=ps[0].index(value)
                grizy=self.readselect(ids,cat='PS')
                #print(grizy)

            #For all 2MASS IDs:
            else:
                k2=list(self.K2)
                ids=k2[3].index(value)
                jhk=self.readselect(ids,cat='2MASS')
                #print(jhk)
        return

In [134]:
def to_xms(df,name, x):

    list_ = df[name].tolist()
    if x == 'h':
        s = [item[:2]+'h'+item[3:5]+'m'+item[6:]+'s' for item in list_]
        s = Angle(s,unit=u.deg)
    elif x == 'd':
        s = [item[:3]+'d'+item[4:6]+'m'+item[7:]+'s' for item in list_]
        s = Angle(s,unit=u.deg)
    elif x == 'udeg':
        s = Angle(list_,unit=u.deg)
        #s = Angle.to_string(list_,sep='dms',pad=True)
    elif x == 'none':
        s = Angle(list_,unit=u.deg)
    
    return pd.Series(s).values
#---------------------------------------------------------------------------------------------------------------------

# def sampedro_match(sampedro,cat, dist='0d0m3s',debug=False):

#     '''

#     Matches Sampedro catalog with a number (n) of membership approvals with catalog (cat) entries in a radius (dist) given in 'XhYmZs'.

#     Output:

#     match_idx: list - indices of objects in Pan-STARRS that match with object positions in Sampedro cluster list 

#     '''
#     #print('This is the sampedro catalog: \n{}\n'.format(sampedro.head()))
#     #print('This is the k2mass catalog: \n{}\n'.format(cat.head()))   
#     ra2 = list(cat.RAJ2000) 
#     dec2 = list(cat.DEJ2000)
    
# #     if cat=='Pan-STARRS':
# #         l=x.PS.copy()
# #         ra2 = l.RAJ2000
# #         dec2 = l.DEJ2000
# #     elif cat == 'K2MASS':
# #         l=x.K2MASS.copy()
# #         ra2 = l.RAJ2000
# #         dec2 = l.DEJ2000
# #     else:
# #         print('This is not a valid catalog chiffre. I\'ll use Pan-STARRS.')
# #         l=x.PS.copy()
# #         ra2 = l.RAJ2000
# #         dec2 = l.DEJ2000

# #     l2=sampedro[n].copy()
#     ra3 = list(sampedro.RAJ2000) 
#     dec3 = list(sampedro.DEJ2000)
#     #match Sampedro cluster members with Pan-STARRS
#     #try:
#     #print('This is the sampedro catalog 2: \n{}\n'.format(sampedro.head()))
#     #print('This is the k2mass catalog 2: \n{}\n'.format(cat.head()))    
#     match_idx=match(sampedro,cat,ra2,dec2,ra3,dec3,dist)
#     #, match_K2ID, match_K2Ra,match_K2Dec,len_match_idx,other_match_ID,match_2mass_ID
# # 		except ValueError:
# # 			print('This is ra PS \n{}\n and this is dec PS \n{}\n'.format(ra2[0:5],dec2[0:5]))
# # 			print(cat + 'catalog list or Sampedro_n' + str(n) + ' are probably empty. No matching possible.')
# # 			match_idx=[0]



#     return match_idx#, match_K2ID, match_K2Ra,match_K2Dec,len_match_idx,match_2mass_ID

def sampedro_match(sampedro, cat, debug=False):
    
    c = pd.merge(cat,sampedro[['UCAC4','RAJ2000','DEJ2000']],on='UCAC4')
    c['del']=np.sqrt((c.RAJ2000_x-c.RAJ2000_y)*(c.RAJ2000_x-c.RAJ2000_y)+(c.DEJ2000_x-c.DEJ2000_y)*(c.DEJ2000_x-c.DEJ2000_y))

    c['UCAC4'] = c.UCAC4
    print('This is the UCAC4 matched catalog: \n{}\n'.format(c.describe()))
    return c
#--------------------------------------------------------------------------------------------

def second_order_match(lK2,lPS,sampedro, distPS='0d0m3s', dist='0d0m3s',debug=False):

    '''

    Matching K2 and Pan-STARRS (conditional on Sampedro cluster membership with (n) shared assessments).

    '''

    intermediate = sampedro_match(lK2, sampedro,debug=debug)
    print('This is intermediate: \n{}\n'.format(intermediate.describe()))
#     ra2=lK2.RAJ2000_x
#     dec2=lK2.DEJ2000_x
#     match_idx_K2=sampedro_match(sampedro,lK2, dist=distPS,debug=debug)
#     print('These are match_idx_K2:\n{}\n'.format(match_idx_K2[:10]))
#     lK2 = lK2.iloc[match_idx_K2]
#     ra2=lK2.RAJ2000
#     dec2=lK2.DEJ2000
    
#     print('From the cluster members as assessed by Sampedro '+ str(len(lK2)) + ' have an EVEREST LC.')
#     ra3=lPS.RAJ2000
#     dec3=lPS.DEJ2000

#     second_match_idx=match(lPS,lK2,ra2,dec2,ra3,dec3,dist)

#     SOM = lK2.iloc[second_match_idx]
    SOM = intermediate.merge(lPS, on='UCAC4', how='inner')
    print('This is the UCAC4 double-matched catalog: \n{}\n'.format(SOM.describe()))
    

    return SOM


#------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------

In [135]:
#General purpose funcs:
def match(sampedro,other_cat,ra1,dec1,ra2,dec2,dist='0d0m1s'):#ra2, dec2 correspond to sampedro
    '''
    Finds the entries in a catalog cat with corresponsing ra2 and dec2 
    that match with a list or coordinates (ra1,dec1)  
    '''
    #print(ra1[0:10],ra2[0:10],dec1[0:10],dec2[0:10])
    #sampedro.RAJ2000 = to_xms(sampedro,'RAJ2000', 'none')
    #sampedro.DEJ2000 = to_xms(sampedro,'DEJ2000', 'none')
    #print(other_cat['RAJ2000'])
    #other_cat.RAJ2000 = to_xms(other_cat,'RAJ2000', 'none')
    #other_cat.DEJ2000 = to_xms(other_cat,'DEJ2000', 'none')
    ra1,ra2,dec1,dec2=Angle(ra1,unit=u.deg),Angle(ra2,unit=u.deg),Angle(dec1,unit=u.deg),Angle(dec2,unit=u.deg)
    #,unit=u.deg)
    #c = ICRS(ra=list(cat.RAJ2000), dec=list(cat.DEJ2000))
    c = ICRS(ra=ra2, dec=dec2)
    #print(c)
    #sampedro['ICRS'] = pd.Series(c)#.values
    print('This is the sampedro catalog in match: \n{}\n'.format(sampedro.head()))
    
    #catalog = ICRS(ra=list(other_cat.RAJ2000), dec=list(other_cat.DEJ2000) )
    catalog = ICRS(ra=ra1, dec=dec1)
    #print(catalog)
    #other_cat['ICRS'] = pd.Series(catalog).values
    print('This is the other catalog in match: \n{}\n'.format(other_cat.head()))
    
    idx, d2d, d3d = match_coordinates_sky(c, catalog,nthneighbor=1) #idx are indices into other cat that are the closest objects to each of the coordinates in c
    second_match_idx = are_within_bounds(idx, d2d,'0d0m0s', dist)
    
    other_idx, other_d2d,other_d3d = match_coordinates_sky(catalog,c,nthneighbor=1)
    other_second_match_idx = are_within_bounds(other_idx, other_d2d,'0d0m0s', dist)
    
    print('These are typical distances to closest neighbors: \n{}\n'.format(d2d[:4]))
    print(second_match_idx[:4],idx[:4])
    print(other_second_match_idx[:4],other_idx[:4])
    
    #match_ID=[i for j, i in enumerate(cat[0]) if j in second_match_idx]
#     try:  
#     	match_2mass_ID=[i for j, i in enumerate(cat[3]) if j in second_match_idx]
#     except IndexError:
#         match_2mass_ID=[]
#     match_Ra=[i for j, i in enumerate(cat[1]) if j in second_match_idx]
#     match_Dec=[i for j, i in enumerate(cat[2]) if j in second_match_idx]
    
#     other_match_ID=[i for j, i in enumerate(other_cat.UCAC4) if j in other_second_match_idx]
    return second_match_idx#,match_ID,match_Ra, match_Dec, str(len(second_match_idx)), other_match_ID,match_2mass_ID


def find(name, path,debug=False):
    
    '''
    Return the path to a file with name if it exists within the folder in path.
    
    Args:
    name: name of the file one is looking for
    path: path where on suspects to find the file
    debug: generates extra output to the command prompt for debugging
    
    Returns:
    path to file
    '''
    
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

def are_within_bounds(idx,d2d, min_angle, max_angle,debug=False):
	
	'''
	Check if the objects with indices idx which match a catalog object within distance d2d lie within [min_angle, max-angle].
	
	Args:
	idx: list of indices into said catalog for identification
	d2d: list of corresponding distances
	min_angle: minimum distance in dms format
	max_angle: maximum distance in dms format
	debug: generates extra output to the command prompt for debugging
	
	Returns:
	n: indices extracted from idx that fulfil the condition for angular distance
	'''
	
	l=[]
	d=[]
	max_angle='0d0m5s'
	#print('So many distances: {}'.format(d2d))
	for _ in range(len(d2d)):	
		if d2d[_].is_within_bounds(min_angle, max_angle): 
			l.append(idx[_])
			d.append(d2d[_])
	print('These are the indices within bounds: \n{}\n'.format(l))
	n=[]
	lpop=list(l)
	dpop=list(d)
	while len(lpop)>0:
		#find same indices
		m=[i for i, j in enumerate(lpop) if j == lpop[0]]
		if len(m)>1:
			for k in range(len(m)-1):
				if dpop[m[k]].is_within_bounds(dpop[m[k+1]]): 
					minidx=lpop[m[k]]
				else: 
					minidx=lpop[m[k+1]]
		else: minidx=lpop[m[0]]
		n.append(minidx)
		lpop=[i for j, i in enumerate(lpop) if j not in m]
		dpop=[i for j, i in enumerate(dpop) if j not in m]
	return n


#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------


def wrap_cross(inputs,debug=True):
	for item in inputs:
		obj=OpenCluster(item[0],item[1], radius=item[2], age=item[3])
		obj.loadcatalogs(debug=debug)
		print('\nMatching catalogs for ' + obj.name + ':\n')
		obj.refinesampedro(debug=debug)
		#with        
		#out.write(str(len(x.sampedro_n0[0]))+'\n'+str(len(x.sampedro_n1[0]))+'\n'+str(len(x.sampedro_n2[0]))+'\n'+str(len(x.sampedro_n3[0]))+'\n'+str(len(x.PS[0]))+'\n'+str(len(x.K2[0]))+'\n')
		SOM = second_order_match(obj.K2,obj.PS,obj.sampedro_n1,distPS='0d0m3s',dist='0d0m3s',debug=debug)
		print('SECOND ORDER MATCH: \n{}\n'.format(SOM.describe()))
        #, ps_m, psra_m,psdec_m,len1,PS_match_ID=
#		print('This is K2MASS before matching: \n{}\n'.format(obj.K2MASS.info()))
#		print('This is sampedro before matching: \n{}\n'.format(obj.sampedro_n1.info())) 
# 		sampedro = obj.sampedro_n1
# 		cat = obj.K2MASS
# 		print('This is K2MASS before matching: \n{}\n'.format(obj.K2MASS.info()))
# 		print('This is sampedro before matching: \n{}\n'.format(obj.sampedro_n1.info()))
# # 		lk2_m, k2_m,k2ra_m,k2dec_m, len2,twomass_match_ID
# 		id_ = sampedro_match(sampedro,cat,dist='0d0m3s',debug=debug)
# 		print('This is K2MASS matched with sampedro: \n{}\n'.format(obj.K2MASS.iloc[id_].head()))    
# 		inter1=open(x.sampedro_name + '_inter1.txt', 'w')#epic,ps
# 		inter2=open(x.sampedro_name + '_inter2.txt', 'w')#epic,2mass
# 		for a,b in zip(ps_m,PS_match_ID):
# 			inter1.write(str(a)+','+str(b)+'\n')
# 		inter1.close()		
# 		for a,b in zip(k2_m,twomass_match_ID):
# 			inter2.write(str(a)+','+str(b)+'\n')
# 		inter2.close()
	return #x.PS, x.K2, x



#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------

In [136]:
inputs=[]
#inputs.append(['M67','M67', 15, 4.0])
#inputs.append(['Ruprecht 147','Ruprecht_147', 30, 2.5, 300.])
inputs.append(['M44','M44', 47, 0.73, 181.5])
wrap_cross(inputs,debug=True)
#PS,K2,X=
#print(type(X))
#wrap_further(X)

Currently working in "/home/ekaterina/Documents/Matching_Catalogs"

These are our K2 data:

        EPIC     RAJ2000    DEJ2000       UCAC4             2MASS
0  211960587  130.139429  19.633600  549-045501  08403347+1938009
1  211959779  130.040342  19.621425  549-045474  08400968+1937170
2  211968598  130.078517  19.754422  549-045484  08401884+1945158
3  211968228  130.057425  19.748875  549-045479  08401378+1944559
4  211958885  130.158300  19.607928         NaN  08403799+1936285

These are our Sampedro data:

        UCAC4  RAJ2000  DEJ2000  ClassM1  ClassM2  ClassM3
0  546-046614  129.729  19.1983        0        1        0
1  546-046624  129.786  19.1915        0        1        0
2  546-046625  129.789  19.1626        0        0        0
3  546-046630  129.808  19.1917        0        0        0
4  546-046634  129.816  19.1891        0        1        0

These are our Pan-STARRS data:

      gmag     rmag     imag     zmag     ymag       UCAC4
0      NaN  19.9185      NaN  21.39

In [35]:
def wrap_basic(inputs,debug=False):
	for item in inputs:
		
		x=OpenCluster(item[0],item[1], radius=item[2], age=item[3])
		x.loadcatalogs(debug=debug)
		print('\nMatching catalogs for ' + x.name + ':\n')
		x.refinesampedro(debug=debug)
		out=open('share/'+item[1]+'_IDs.txt', 'w')
		out.write(str(len(x.sampedro[0]))+'\n'+str(len(x.sampedro_n1[0]))+'\n'+str(len(x.sampedro_n2[0]))+'\n'+str(len(x.sampedro_n3[0]))+'\n'+str(len(x.PS[0]))+'\n'+str(len(x.K2[0]))+'\n')
		

		#all line numbers +2 in parameters sheet
		for i in range(1,4):
			##10
			l, k2, ra,dec,length=x.sampedro_match(i,dist='0d0m5s',cat='Pan-STARRS',debug=debug)
			out.write(length+'\n')
			##11
			l, k2, ra,dec,length=x.sampedro_match(i,dist='0d0m3s',cat='Pan-STARRS',debug=debug)
			out.write(length+'\n')
		for i in range(1,4): 
			##16,18,20
			l, k2, ra,dec,length=x.sampedro_match(i,dist='0d0m5s',cat='K2MASS',debug=debug)
			out.write(length+'\n')
			##17,19,21
			l, k2, ra,dec,length=x.sampedro_match(i,dist='0d0m3s',cat='K2MASS',debug=debug)
			out.write(length+'\n')
		
			# if i==1:		
			#     for item in ps:
			#         ids.write(str(item)+'\n')
		for i in range(1,4):
			##22,26,30
			l, ps, ra,dec,length,other_match_ID=x.second_order_match(i,distPS='0d0m5s',dist='0d0m5s',debug=debug)
			out.write(length+'\n')
			##23,27,31
			l, ps, ra,dec,length,other_match_ID=x.second_order_match(i,distPS='0d0m5s',dist='0d0m3s',debug=debug)
			out.write(length+'\n')
			##24,28,32
			l, ps, ra,dec,length,other_match_ID=x.second_order_match(i,distPS='0d0m3s',dist='0d0m5s',debug=debug)
			out.write(length+'\n')
			##25,29,33
			l, ps, ra,dec,length,other_match_ID=x.second_order_match(i,distPS='0d0m3s',dist='0d0m3s',debug=debug)
			out.write(length+'\n')
			
			#out.write(length+'\n')
		out.close()
	return

In [36]:
import pandas as pd
def readselect2(PS,K2,obj,ids,cat='PS'):

    import linecache
    cats={'PS':['_panstarrs_search.txt',[24,32,40,48,56]]}
    dt=np.unicode_
    line=linecache.getline('cats/'+obj.sampedro_name+cats[cat][0],ids)
    #print(line)
    myarray=np.core.defchararray.split(line, sep="\t").tolist()
    
    
    print(myarray)
    mags=[float(myarray[i]) for i in cats[cat][1]]
    
    return mags
#--------------------------------------------------------------------------------------------

def read_params2(PS,obj,udict):
    k2=np.genfromtxt('share/cross_match/union/'+obj.sampedro_name + '_k2_JHK.csv',
                     delimiter=',',
                     skip_header=2,
                     usecols=(0,20,27,28,29),dtype=[('k2ID','U12'),('ID','U16'),('J','f8'),('H','f8'),('K','f8')])
    df = pd.read_csv('share/cross_match/union/'+obj.sampedro_name + '_k2_JHK.csv',
                     delimiter=',',
                     skiprows =[1],
                     usecols=['K2 ID','2MASS ID','J Mag','H Mag','K Mag'],
                    )
    ps = pd.read_csv('cats/'+obj.sampedro_name + '_panstarrs_search.txt',
                     delimiter=',',
                     skiprows =[1],
                     usecols=['K2 ID','2MASS ID','J Mag','H Mag','K Mag'],
                    )
    df.rename(index=int,
              columns={'K2 ID': 'EPIC','2MASS ID':'2MASS','J Mag' : 'J','H Mag' : 'H','K Mag': 'K'},
              inplace=True
             )
    #ps = pd.DataFrame(list(PS),columns=['PSID'])
    print('\n',df.head())
    #J=k2['J'].tolist()
    #H=k2['H'].tolist()
    #K=k2['K'].tolist()
    #k2mass=k2['ID'].tolist()
    #k2ID=k2['k2ID'].tolist()
    #k2ID=[float(item) for item in k2ID]
  
    #ps=list(PS)
    u={key : [udict[key]] for key in udict}
    union = pd.DataFrame.from_dict(u).T
    union.rename(index=int, columns={0 : 'id'},inplace=True)
    #print(union.head())
    udict={}
    for epic, id_ in union.iterrows():
        print(type(id_.id))
        #For all Pan-STARRS IDs:
        if type(id_.id) == int:

            #ids=ps[0].index(value)
            grizy=readselect2(PS,K2,obj,epic,cat='PS')
            grizy.append(df.J[df.EPIC==epic])
            #print(grizy)
            
            udict[epic]=[id_,'grizyJ',grizy]
            

        #For all 2MASS IDs:
        else:
            id_=str(id_)
            for ids, row in df.iterrows():
                if row['2MASS'] == id_:
                    jhk=[df.J[ids],df.H[ids],df.K[ids],df.J[ids]]
                    #print(jhk)
                    udict[epic]=[id_,'JHKJ',jhk]
    #print(udict)
    return udict

In [524]:

x=OpenCluster('Ruprecht 147','Ruprecht_147', 30, 2.5, 300.)
x.loadcatalogs(debug=True)
x.refinesampedro(debug=True)
#x.PS.describe()

Currently working in "/work1/eilin/data/CLUSTERS_01/k2-panstarrs-sampedro"

These are our K2 data:

        EPIC        RAJ2000        DEJ2000             2MASS
0  219622634  19h16m40.735s  -16d16m41.06s  19164073-1616411
1  200062550  19h16m37.246s  -16d16m27.69s               NaN
2  200062557  19h16m49.285s  -16d18m37.68s               NaN
3  219605951  19h16m46.628s  -16d19m20.93s  19164662-1619208
4  200062558  19h16m36.568s  -16d19m24.91s               NaN

These are our Sampedro data:

        UCAC4       RAJ2000        DEJ2000  ClassM1  ClassM2  ClassM3
0  368-169970  289d03m32.4s  -16d24m19.08s        0        1        0
1  368-169975  289d03m39.6s   -16d24m07.2s        1        0        1
2  368-169983  289d03m43.2s     -16d25m03s        0        1        0
3  368-169996    289d04m12s   -16d24m48.6s        0        1        0
4  368-170000  289d04m19.2s  -16d25m19.56s        0        1        0

These are our Pan-STARRS data:

               objID       RAJ2000       DEJ2000  

,objID,gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag
count,5.000100e+04,50001.000000,50001.000000,50001.000000,50001.000000,50001.000000
mean,8.846369e+16,-391.931991,-235.806967,-66.245731,-149.803012,-294.673510
std,1.518346e+14,500.140587,441.735405,283.236166,378.999409,469.889681
min,8.815289e+16,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
25%,8.834289e+16,-999.000000,-999.000000,18.391000,17.343000,-999.000000
50%,8.846289e+16,18.529000,19.427000,19.942000,19.356000,18.333000
75%,8.858289e+16,20.800000,20.899000,20.997000,20.395000,19.589000
max,8.876289e+16,27.631000,24.969000,25.172000,22.000000,21.425000


In [4]:
def wrap_further(X):
    ps_m,PS_match_ID=np.loadtxt(X.sampedro_name + '_inter1.txt', delimiter=',', unpack=True,dtype=[('ID','i8'),('PS', 'i8')])
    k2_m,twomass_match_ID=np.loadtxt(X.sampedro_name + '_inter2.txt', delimiter=',', unpack=True,dtype=[('ID','i8'),('2MASS',np.unicode_,18)])
    ps_m,PS_match_ID,k2_m,twomass_match_ID=list(ps_m),list(PS_match_ID),list(k2_m),list(twomass_match_ID)
    
    ps = pd.read_csv(X.sampedro_name + '_inter1.txt', 
                     delimiter=',',
                     names=['EPIC','PSID']
                    )
    k2 = pd.read_csv(X.sampedro_name + '_inter2.txt', 
                     delimiter=',',
                     names=['EPIC','2MASS']
                    )
    print(ps.head(),'\n',k2.head())

    PSdict=dict(zip(ps_m,PS_match_ID))
    TwoMASSdict=dict(zip(k2_m,twomass_match_ID))
    cross=set(ps_m) & set(k2_m)
    union=set(ps_m) | set(k2_m)
    union_dict = TwoMASSdict
    union_dict.update(PSdict)
    #print(PSdict)
    #print(union_dict)
    #print(len(union))
    #print(type(PS_match_ID[0]))

    udict_color=read_params2(PS,X,union_dict)
    
    with open('share/cross_match/union/'+X.sampedro_name+'_parameter.txt', 'w') as udict_out:
        udict_out.write(X.sampedro_name + ', ' + str(X.radius) + ' arcmin, ' + str(X.age) + ' Gyr, ' + str(X.distance) + ' pc\n')
        for key, value in udict_color.items():
            color=''
            if value[1] == 'grizyJ':
                for i,col in enumerate(value[2]):
                    color=color+','+str(col)
                udict_out.write(str(key)+','+str(value[0])+','+str(value[1])+color+',NaN,NaN\n')
            elif value[1] == 'JHKJ':
                    for i,col in enumerate(value[2][:-1]):
                        color=color+','+str(col)
                        
                    udict_out.write(str(key)+','+str(value[0])+','+str(value[1])+',NaN,NaN,NaN,NaN,NaN'+color+'\n')



    #Objects with 2MASS and Pan-STARRS respectively
    id2mass=open('share/2mass_match/'+X.sampedro_name+'_2mass_IDs.txt', 'w')
    idps=open('share/panstarrs_match/'+X.sampedro_name+'_panstarrs_IDs.txt', 'w')
    for i in k2_m:
        id2mass.write('EPIC '+str(i)+'\n')

    for i in ps_m:
        idps.write('EPIC '+str(i)+'\n')
    idps.close()
    id2mass.close()


    #Cross section of Pan-STARRS and 2MASS
    idcross=open('share/cross_match/'+X.sampedro_name+'_cross_IDs.txt', 'w')
    #Union of Pan-STARRS and 2MASS		
    idunion=open('share/cross_match/union/'+X.sampedro_name+'_union_IDs.txt', 'w')	
    for i in list(cross):
        idcross.write('EPIC '+str(i)+'\n')

    for i in union:
        idunion.write('EPIC '+str(i)+'\n')

    idcross.close()
    idunion.close()
    return